# We get all the modules necessary for the simulation

In [1]:
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from numpy.random import randint
import numpy as np

# We now initialize a random seed to get our initial strings of lenght n

In [2]:
np.random.seed(seed=0)
n=128  #number of bits
aliceBits = randint(2, size=n)
bits=''.join(str(x) for x in aliceBits)
aliceBasis = randint(2, size=n)
basis=''.join(str(x) for x in aliceBasis)


# We have the bits we want to sent

In [3]:
bits

'01101111111001000001011001111010101101100101111101011110100110101000001100011010010111111011001001101001000110100000101011111011'

# and the basis in which to encode our bits

In [4]:
basis

'11011001000011001011110001011101001011001010101010001010100000100100010010100110001100000101000111001111000110100101111000111011'

# We then encode the basis by preparing the state in either the X or Y basis based on the initial random string

In [5]:
def encodeMessage(bits, basis):
    message = []
    for i in range(n):
        qc = QuantumCircuit(1,1)
        
        if basis[i] == 0: # 0 means we prepare and send in the Z-basis
            4 if bits[i] == 0 else qc.x(0) # If the bit is 0 we do nothing as circuits are initialized in the |0> state
                                       #the other option is that the bit is 1 then we use an X gate to convert to the |1> state
        else:                          # Prepare qubit in X-basis
            4 if bits[i]==0 else qc.x(0)  # if 0 we send the |+> if not the |->
            qc.h(0)
        qc.barrier()
        message.append(qc)
    return message

In [6]:
message = encodeMessage(aliceBits, aliceBasis)

# Then measure chooses the basis in which he will measure at random as he receives Alice's qubits

In [7]:
bobBasis = randint(2, size=n)
def measureMessage(message, basis):
    measurements = []
    for i in range(n):
        if basis[i] == 0: # measuring in Z-basis
            message[i].measure(0,0)
        if basis[i] == 1: # measuring in X-basis
            message[i].h(0)
            message[i].measure(0,0)
        sim = Aer.get_backend('qasm_simulator')
        gates = assemble(message[i], shots=1, memory=True) # just the one measure 
        result = sim.run(gates).result()
        measuredBit = int(result.get_memory()[0])
        measurements.append(measuredBit)
    return measurements

In [8]:
bobResults = measureMessage(message, bobBasis)


# Then Alice announces the basis and her and Bob discard all bits in which they used a different basis

In [9]:
def discardDifferent(aliceBasis, bobBasis, bits):
    kept = []
    for i in range(n):
        if aliceBasis[i] == bobBasis[i]:
            kept.append(bits[i])
    return kept

In [10]:
aliceKey = discardDifferent(aliceBasis, bobBasis, aliceBits)
bobKey = discardDifferent(aliceBasis, bobBasis, bobResults)

# Now both have their keys since no eavesdropper  is present here the keys should be identincal

In [12]:
aliceKey==bobKey

True

# The next step would be  sampling the bits to look for an eavesdropper, however let us introduce one in the simulation before

In [14]:
# she decides to measure the basis at random
eveBasis = randint(2, size=n)
interceptedMessage = measureMessage(message, eveBasis)




In [16]:
print(interceptedMessage)

[0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0]


In [21]:
def resend(intercepted):
    message=[]
    for i in range(n):
        qc = QuantumCircuit(1,1)
        0 if bits[i] == 0 else qc.x(0) 
        qc.barrier()
        message.append(qc)
    return message

In [22]:
newmessage=resend(interceptedMessage)

In [23]:
bobResults = measureMessage(newmessage, bobBasis)

# Now that there was an eavesdropper when they discard the different basis, their keys are different

In [46]:
aliceKey = discardDifferent(aliceBasis, bobBasis, aliceBits)
bobKey = discardDifferent(aliceBasis, bobBasis, bobResults)

In [49]:
def checkBits(bits, selection):
    checkSample = []
    for i in selection:
        i = i % len(bits)
        checkSample.append(bits.pop(i))
    return checkSample

In [50]:
sampleSize = 20
selection = randint(n, size=samplSize)


In [51]:
bobSample = checkBits(bobKey,selection)
aliceSample=checkBits(aliceKey,selection)

In [52]:
bobSample

[1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0]

In [53]:
aliceSample

[1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1]

# As the samples are too different an eavesdropper is infered and the protocol needs to be repeated

In [54]:
len(bobSample) #roughly half of them are mistaken, as the 50% chance in the x basis

20

In [48]:
len(aliceKey)

61

In [56]:
count=0
for i in range(len(aliceKey)):
    if aliceKey[i]==bobKey[i]:
        count+=1
    

In [57]:
count

27